In [1]:
import numpy as np
import scipy.integrate as integrate
%pylab inline

from swap import Swap,priceSwap
from marketSetup import simulateOIS,simulateSurvivalProb

Populating the interactive namespace from numpy and matplotlib


### 3 CVA/DVA Exercise

###### Simulate OIS curve

From equation (18) and (19) from [1], we get $$\sigma_{22}=\sigma_2$$
$$\sigma_{21}=\sigma_1 \times \rho_x$$
$$\sigma_{11}=\sqrt{\sigma_1^2-\sigma_{21}^2}=\sqrt{\sigma_1^2-\sigma_1^2\times \rho_x^2}$$
For $y(t)$, we calculate the involved integral manually:
$$\int_{0}^{t}{a(u)^T a(u)du}=\int_{0}^{t} \begin{bmatrix} \sigma_{11}^2e^{2\kappa_1u} & \sigma_{21}\sigma_{22}e^{(\kappa_1+\kappa_2)u}\\\sigma_{21}\sigma_{22}e^{(\kappa_1+\kappa_2)u} & \sigma_{22}^2e^{2\kappa_2u} \end{bmatrix}du \\=\begin{bmatrix} \frac{\sigma_{11}^2+\sigma_{21}^2}{2\kappa_1}(e^{2\kappa_1t}-1) & \frac{\sigma_{21}\sigma_{22}}{\kappa_1+\kappa_2}(e^{(\kappa_1+\kappa_2)t}-1) \\ \frac{\sigma_{21}\sigma_{22}}{\kappa_1+\kappa_2}(e^{(\kappa_1+\kappa_2)t}-1) & \frac{\sigma_{22}^2}{2\kappa_2}(e^{2\kappa_2t}-1)\end{bmatrix}$$
For $P(t,x(t),T)$, we know $$P(t,x(t),T)=e^{-\int_{t}^{T} f(t,x(t),u)du}$$ where $$f(t,x(t),u)=f(0,u)+M(t,u)^T(x(t)+y(t)G(t,u))$$
Since $f(0,u)$ is constant, then $$P(t,x(t),T)=e^{-f(0,T)+f(0,t)}e^{-x(t)\int_{t}^{T}M(t,u)du}e^{-y(t)\int_{t}^{T}M(t,u)^TG(t,u)du}\\=\frac{P(0,T)}{P(0,t)}exp\big(-G(t,T)^Tx(t)-\frac{1}{2}G(t,T)^Ty(t)G(t,T)\big)$$
where $$M(t,T)=\begin{bmatrix} e^{-\kappa_1(T-t)} \\ e^{-\kappa_2(T-t)}\end{bmatrix}$$
and $$G(t,T)=\int_{t}^{T}M(t,u)du=\int_{t}^{T}\begin{bmatrix} e^{-\kappa_1(T-t)} \\ e^{-\kappa_2(T-t)}\end{bmatrix}du\\=\begin{bmatrix} \frac{1-\kappa_1(T-t)}{\kappa_1} \\ \frac{1-\kappa_2(T-t)}{\kappa_2}\end{bmatrix}$$

###### Inputs

In [2]:
num_simulation = 50000
sim_freq = 12

### Interest rate parameters
spread = 0.005 # spread i.e. f_LIBOR=f_OIS+spread
f0_OIS = 0.02
f0_LIBOR = f0_OIS+spread

### swap parameters
freq = 2
maturity = 10
coupon = 0.0265
notional = 150000000.

### Interest rate model parameters
sigma_r = 0.02
c = 0.35
kappa1,kappa2 = 0.02,0.1
rho_inf = 0.4
nu = np.sqrt(1./c/c - 1. - 2.*(rho_inf/c - 1.))
rho_x = (rho_inf/c - 1.)*nu
sigma_l = c * sigma_r
sigma1 = sigma_l
sigma2 = nu*sigma1

### Credit curve parameters
lbda0_B, lbda0_C = 0.01,0.03
sigmaB, sigmaC = 0.005,0.01
kappaB, kappaC = 0.1,0.1

rho_Bf, rho_Cf = 0.1,0.1
rho_BC = 0.75
rho_Br, rho_Cr = 0.25,0.25
rho_B1 = rho_Bf   # corr b/w lbdaB and x1
rho_C1 = rho_Cf   # corr b/w lbdaC and x1
rho_B2 = rho_Br*np.sqrt(nu*nu+1.+2*rho_x*nu)-nu*rho_B1   # corr b/w lbdaB and x2
rho_C2 = rho_Cr*np.sqrt(nu*nu+1.+2*rho_x*nu)-nu*rho_C1   # corr b/w lbdaC and x2

### correlation matrix among lbdaB,lbdaC,x1,x2 for simulation
corr = np.array([[1., rho_BC, rho_B1, rho_B2],\
                 [rho_BC, 1., rho_C1, rho_C2],\
                 [rho_B1, rho_C1, 1., rho_x],\
                 [rho_B2, rho_C2, rho_x, 1.]])
chol = np.linalg.cholesky(corr)

In [5]:
swap = Swap(maturity, coupon, freq, notional)
swap.__str__()

Tis = np.arange(1./freq,maturity+1e-6,1./freq)
ts = np.arange(1./sim_freq,maturity+1e-6,1./sim_freq)
prices=[]

for num in range(num_simulation):
    # simulate correlated 4-D brownian motion
    wt = chol.dot(np.random.normal(0,1./sim_freq,(4,sim_freq*maturity)))
    
    P_OIS, P_LIBOR = simulateOIS(rho_x, sigma1, sigma2, kappa1, kappa2, sim_freq, maturity, f0_OIS, spread,ts,Tis,wt)
    X_B,X_C,lbdaB,lbdaC = simulateSurvivalProb(lbda0_B,lbda0_C,ts,sigmaB,kappaB,sigmaC,kappaC,wt)
    price_one_path=[]
    for i in range(maturity*sim_freq):
        price_one_path.append(priceSwap(swap, 'receiver', P_OIS, P_LIBOR, i, ts,Tis, sim_freq))
        
    prices.append(price_one_path)

In [9]:
print np.average(np.asarray(prices),axis=0)

[ 17935555.74399479  18223244.98544963  18651833.80792002  18774362.1939109
  18102355.89666903  18253948.52592488  17103477.28053838
  17234622.10630935  16700780.09671983  16717462.00519546
  17011673.94081719  16714291.32105072  16012376.97456519
  16613792.91190851  16215600.64965993  15678416.54045297
  15417172.31719364  15980138.01705843  15291061.6946653   15011213.20030542
  15249396.50453481  14476372.65029493  13703057.56510778
  14096928.69113078  13415056.13279616  13197373.52149249
  13044209.38540511  12990035.2754441   12843018.82998998
  13006239.44743251  12432691.97835331  12051914.74355435
  11718888.27038207  11708828.1929338   11693157.46761988
  11537619.63205654  10654864.61925268  10909241.27740008
  11177878.94387699  11247720.89475487  10762793.6611129   10611294.14243466
   9762844.48056854   9710997.67652716   9976901.80863079
   9991012.02422942   9789332.20461347  10004290.69416536
   9377872.33873739   8962117.26825208   8348513.54722665
   8108024.82042

In [ ]:
wt = chol.dot(np.random.normal(0,1./sim_freq,(4,sim_freq*maturity)))
ts = np.arange(1./sim_freq,maturity+1e-6,1./sim_freq)
X_B,X_C,lbdaB,lbdaC = marketSetup.simulateSurvivalProb(lbda0_B,lbda0_C,ts,sigmaB,kappaB,sigmaC,kappaC,wt)
#plot(ts,X_B,ts,X_C)
print len(X_C)
print len(X_C[1])

In [ ]:
a = np.array([0,1,2,3,4,5,6,7,8,9,10,11,13])
np.where(a[::3]>3)
a[0::3]
np.argmax(a>3)

In [13]:
a = np.array([0,1,2,-3,4])
print np.maximum(a,0)

[0 1 2 0 4]


In [ ]:
plot(np.average(prices,axis=0));

In [ ]:
np.random.normal(0,1,(4,3))

###### Credit Curve Setting

In [ ]:
# test cell
rho_x = 0.5
C = np.asarray([[1,0],[rho_x,1-rho_x*rho_x]]) # Cholesky decomposition of correlation matrix
wt = C.dot(np.random.normal(0,1,(2,)))
print wt.shape
print np.ones(2).shape
print np.diag(np.asarray([2,3]))
print np.arange(1./12,10.00001,1./12)

In [ ]:
print len(P_OIS[1])
print len(Tis)
print len(P_LIBOR[61])

### 4 IMM Exercise